### LSTM

In [ ]:
li_companies = ['TCS', 'HUL', 'INFY', 'SAIL', 'MRF', 'SRF', 'PVR.NS', 'IDBI.NS', 'RITES.NS']

for company in li_companies:
    
    print("Company : ", company)

    import yfinance as yf
    import numpy as np
    import pandas as pd
    data = yf.download(company, start="2015-01-01")
    data = data.dropna()

    df1=data.reset_index()['Close']

    import matplotlib.pyplot as plt
    plt.plot(df1)

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0,1))
    df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))

    ##splitting dataset into train and test split
    training_size = int(len(df1)*0.65)
    test_size = len(df1)-training_size
    train_data,test_data = df1[0:training_size,:],df1[training_size:len(df1),:1]

    # convert an array of values into a dataset matrix
    def create_dataset(dataset, time_step):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)

    time_step = 100
    X_train, Y_train = create_dataset(train_data, time_step)
    X_test, Y_test = create_dataset(test_data, time_step)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] , 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] , 1)

    Y_train = Y_train.reshape(Y_train.shape[0], 1)
    Y_test = Y_test.reshape(Y_test.shape[0], 1)

    ### Create the Stacked LSTM model
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import LSTM, GRU
    from keras import Model

    model_lstm=Sequential()
    model_lstm.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
    model_lstm.add(LSTM(50,return_sequences=True))
    model_lstm.add(LSTM(50))
    model_lstm.add(Dense(1))
    model_lstm.compile(loss='mean_squared_error',optimizer='adam')

    model_lstm.summary()

    model_lstm.fit(X_train, Y_train, validation_data = (X_test,Y_test), epochs=10, batch_size=64, verbose=1)
    model_lstm.save('Stock_pred_model_lstm.h5')
    
    ### Lets Do the prediction and check performance metrics
    train_predict=model_lstm.predict(X_train)
    test_predict=model_lstm.predict(X_test)

    ##Transformback to original form
    train_predict=scaler.inverse_transform(train_predict)
    test_predict=scaler.inverse_transform(test_predict)

    import math
    import numpy
    from sklearn.metrics import mean_squared_error

    print("Train Error LSTM : ", math.sqrt(mean_squared_error(Y_train,train_predict)))
    print("Test Error LSTM : ", math.sqrt(mean_squared_error(Y_test,test_predict)))

    ### Plotting 
    look_back=100
    trainPredictPlot = numpy.empty_like(df1)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
    testPredictPlot = numpy.empty_like(df1)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict

    plt.plot(scaler.inverse_transform(df1))
    plt.plot(trainPredictPlot)
    plt.plot(testPredictPlot)

    x_input = test_data[X_test.shape[0]:].reshape(1,-1)
    temp_input = list(x_input)
    temp_input = temp_input[0].tolist()

    # demonstrate prediction for next no of defined days
    from numpy import array

    lst_output = []
    n_steps = 100
    i = 0
    no_of_days = 30

    while(i < no_of_days):

        if(len(temp_input)>100):
            x_input = np.array(temp_input[1:])
            x_input = x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            yhat = model_lstm.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            lst_output.extend(yhat.tolist())
            i=i+1

        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model_lstm.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            lst_output.extend(yhat.tolist())
            i=i+1

    final_out = scaler.inverse_transform(lst_output)
    d = {'Output' : list(final_out)}
    final_df = pd.DataFrame(d) 
    
    import os 
    if os.path.isdir(company):
        final_df.to_csv(company + '/' + company + '_lstm.csv')
        plt.savefig(company + '/' + company + '_lstm.jpg')
        plt.show()
    
    else:
        os.mkdir(company)
        final_df.to_csv(company + '/' + company + '_lstm.csv')
        plt.savefig(company + '/' + company + '_lstm.jpg')
        plt.show()
        
    print("------------------------------------------------------------------")

### GRU

In [ ]:
li_companies = ['TCS', 'HUL', 'INFY', 'SAIL', 'MRF', 'SRF', 'PVR.NS', 'IDBI.NS', 'RITES.NS']
    
final_li = []
for company in li_companies:
    
    print("Company : ", company)

    import yfinance as yf
    import numpy as np
    import pandas as pd
    data = yf.download(company, start="2015-01-01")
    data = data.dropna()

    df1=data.reset_index()['Close']

    import matplotlib.pyplot as plt
    plt.plot(df1)

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0,1))
    df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))

    ##splitting dataset into train and test split
    training_size = int(len(df1)*0.65)
    test_size = len(df1)-training_size
    train_data,test_data = df1[0:training_size,:],df1[training_size:len(df1),:1]

    # convert an array of values into a dataset matrix
    def create_dataset(dataset, time_step):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)

    time_step = 100
    X_train, Y_train = create_dataset(train_data, time_step)
    X_test, Y_test = create_dataset(test_data, time_step)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] , 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] , 1)

    Y_train = Y_train.reshape(Y_train.shape[0], 1)
    Y_test = Y_test.reshape(Y_test.shape[0], 1)
    
    ### Create the Stacked LSTM model
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import LSTM, GRU
    from keras import Model

    model_gru=Sequential()
    model_gru.add(GRU(50,return_sequences=True,input_shape=(100,1)))
    model_gru.add(GRU(50,return_sequences=True))
    model_gru.add(GRU(50))
    model_gru.add(Dense(1))
    model_gru.compile(loss='mean_squared_error',optimizer='adam')

    model_gru.summary()

    model_gru.fit(X_train, Y_train, validation_data = (X_test,Y_test), epochs=10, batch_size=64, verbose=1)
    model_gru.save('Stock_pred_model_gru.h5')

    ### Lets Do the prediction and check performance metrics
    train_predict=model_gru.predict(X_train)
    test_predict=model_gru.predict(X_test)

    ##Transformback to original form
    train_predict=scaler.inverse_transform(train_predict)
    test_predict=scaler.inverse_transform(test_predict)

    ### Calculate RMSE performance metrics
    import math
    import numpy
    from sklearn.metrics import mean_squared_error
    print("Train Error : ", math.sqrt(mean_squared_error(Y_train,train_predict)))
    print("Test Error  : ", math.sqrt(mean_squared_error(Y_test,test_predict)))

    ### Plotting 
    look_back=100
    trainPredictPlot = numpy.empty_like(df1)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
    testPredictPlot = numpy.empty_like(df1)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict

    plt.plot(scaler.inverse_transform(df1))
    plt.plot(trainPredictPlot)
    plt.plot(testPredictPlot)


    x_input = test_data[X_test.shape[0]:].reshape(1,-1)
    temp_input = list(x_input)
    temp_input = temp_input[0].tolist()

    # demonstrate prediction for next no of defined days
    from numpy import array

    lst_output = []
    n_steps = 100
    i = 0
    no_of_days = 30

    while(i < no_of_days):

        if(len(temp_input)>100):
            x_input = np.array(temp_input[1:])
            x_input = x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            yhat = model_gru.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            lst_output.extend(yhat.tolist())
            i=i+1

        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model_gru.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            lst_output.extend(yhat.tolist())
            i=i+1

    final_out = scaler.inverse_transform(lst_output)
    d = {'Output' : list(final_out)}
    final_df = pd.DataFrame(d) 
    
    import os 
    if os.path.isdir(company):
        final_df.to_csv(company + '/' + company + '_gru.csv')
        plt.savefig(company + '/' + company + '_gru.jpg')
        plt.show()
    
    else:
        os.mkdir(company)
        final_df.to_csv(company + '/' + company + '_gru.csv')
        plt.savefig(company + '/' + company + '_gru.jpg')
        plt.show()

    
    print("--------------------------------------------------------------------------")

### AUTO-ARIMA

In [ ]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima

li_companies = ['TCS', 'HUL', 'INFY', 'SAIL', 'MRF', 'SRF', 'PVR.NS', 'IDBI.NS', 'RITES.NS']

for company in li_companies:
    
    print("Comapny : " + company)

    import yfinance as yf
    import numpy as np
    import pandas as pd
    data = yf.download(company, start="2015-01-01")
    df = data.dropna()

    train = df[:int(df.shape[0]*0.75)]
    valid = df[int(df.shape[0]*0.75):]

    training = train['Close']
    validation = valid['Close']

    model = auto_arima(training, trace=True, error_action='ignore', suppress_warnings=True)
    model.fit(training)

    forecast = model.predict(n_periods=len(valid))
    forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])

    rms=np.sqrt(np.mean(np.power((np.array(valid['Close'])-np.array(forecast['Prediction'])),2)))
    print("rms : ", rms)

    import matplotlib.pyplot as plt
    plt.plot(train['Close'])
    plt.plot(valid['Close'])
    plt.plot(forecast['Prediction'])

    import os 
    if os.path.isdir(company):
        forecast.to_csv(company + '/' + company + '_arima.csv')
        plt.savefig(company + '/' + company + '_arima.jpg')
        plt.show()
    
    else:
        os.mkdir(company)
        forecast.to_csv(company + '/' + company + '_arima.csv')
        plt.savefig(company + '/' + company + '_arima.jpg')
        plt.show()
    
    print("----------------------------------------------------------------------------------")